In [1]:
from sklearn.cross_validation import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier,ExtraTreesRegressor
from sklearn.linear_model import LogisticRegression
import pandas as pd
import numpy as np

In [2]:
# Reading input train and test data
train = pd.read_csv("train.csv")
test=pd.read_csv("test.csv")


# removing the constant columns
remove = []
for col in train.columns:
    if train[col].std() == 0:
        remove.append(col)
        
train.drop(remove, axis=1, inplace=True)
test.drop(remove, axis=1, inplace=True)

# remove duplicated columns
remove = []
c = train.columns
for i in range(len(c)-1):
    v = train[c[i]].values
    for j in range(i+1,len(c)):
        if np.array_equal(v,train[c[j]].values):
            remove.append(c[j])

train.drop(remove, axis=1, inplace=True)
test.drop(remove, axis=1, inplace=True)


target = train['TARGET'].values
train = train.drop(['ID','TARGET'], axis=1)

id_test = test['ID'].values
test = test.drop(['ID'], axis=1)

# length of dataset
len_train = len(train)
len_test  = len(test)

In [3]:
# feature engineering
len(train.columns)

# Functions,code for feature engineering

def getCounts(x):
    count=0
    for i in x:
        if i==0:
            count=count+1
    return count
#sum of zeros across the instances
train['zeroCounts']=train.apply(getCounts,axis=1)
test['zeroCounts']=test.apply(getCounts,axis=1)


In [5]:
# Creating a GBT classifier 
from sklearn import metrics
from sklearn import cross_validation
gbt= GradientBoostingClassifier(learning_rate=0.01, subsample=0.8, max_depth=5,max_features=250,n_estimators=160,
                                random_state=1,verbose=1)

In [ ]:
# checking the cross validation for the classifier
scores=cross_validation.cross_val_score(gbt,train,target,cv=5,scoring='roc_auc')
print scores
print("Accuracy: %f (+/- %f)" % (scores.mean(), scores.std() * 2))

In [6]:
# Fit the classifier on the train data
gbt.fit(train,target)


# Predicting on the test data
probs = gbt.predict_proba(test)

      Iter       Train Loss      OOB Improve   Remaining Time 
         1           0.3320           0.0016            3.57m
         2           0.3308           0.0015            3.58m
         3           0.3278           0.0015            3.59m
         4           0.3241           0.0014            3.58m
         5           0.3253           0.0013            3.57m
         6           0.3224           0.0013            3.57m
         7           0.3263           0.0011            3.55m
         8           0.3195           0.0012            3.54m
         9           0.3223           0.0012            3.53m
        10           0.3218           0.0010            3.50m
        20           0.3090           0.0008            3.26m
        30           0.3007           0.0006            3.03m
        40           0.2956           0.0005            2.79m
        50           0.2892           0.0004            2.56m
        60           0.2858           0.0003            2.32m
       

In [7]:
# Forming the submission dataframe
submission = pd.DataFrame({"ID":id_test, "TARGET": probs[:,1]})
submission.to_csv("submission_santander.csv", index=False)